### In this lab exercise, we will become familiar with regression analysis. We will learn how to do descriptive regression analysis in python using the statsmodels library.

### Reference:
- More information on formula syntax: https://patsy.readthedocs.io/en/latest/formulas.html
- More information about statsmodels: https://www.statsmodels.org/dev/example_formulas.html

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

# set seed for consistency
np.random.seed(2)

# Data - preview

We will use a dataset with information about patients with cardiovascular diseases. Here is the breakdown of columns:

Some variables are categorical:
- 'DEATH_EVENT': if the patient deceased during the follow-up period
- 'sex': male/female (binary, male 1, female 0)
- 'anaemia': decrease of red blood cells (boolean, yes 1)
- 'smoking': if the patient smokes or not (boolean, yes 1)
- 'diabetes': if the patient has diabetes (boolean, yes 1)
- 'high_blood_pressure': if the person has hypertension (boolean, yes 1)


... and some are continuous/discrete:
- 'age': patient's age
- 'creatinine_phosphokinase': level of CPK enzyme in the blood (mcg/L)
- 'ejection_fraction': percentage of blood leaving the heart in each contraction
- 'platelets': platelets in the blood (kiloplatelets/mL)
- 'serum_creatinine': level of serum creatinine in the blood (mg/dL)
- 'serum_sodium': level of serum sodium in the blood (mEq/L)
- 'time': duration of the follow-up period (number of days in the hospital)

In [2]:
df = pd.read_csv('data/heart_failure_clinical_records_dataset.csv')
df

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


# Linear regression: Modeling days spent in hospital

- Using linear regression, we will model the number of days spent in the hospital from the patient population.


- We need two components to do that:
   1. An equation describing a model
   2. Data
   

- Equations are specified using [syntax] (https://patsy.readthedocs.io/en/latest/formulas.html) patsy software package:
    1. `~`: Separates the left and right sides
    2. `+`: Creates a union of predictors used
    3. `:`: Interaction term
    3. `*`: `a * b` is a shorter notation for` a + b + a: b`, and is often useful when we want to include all interactions between a set of variables
    
    
- Interceptions are added automatically
- Category variables are added using the term C(a).
- We can use pandas for data!


### Example

- Let's start with an example from our dataset. We are interested in two predictors: diabetes and high blood pressure. These are the two predictors we want to use to fit the outcome, the number of days spent in the hospital, using linear regression.

- The model that achieves this is formulated as:
        time ~ C(diabetes) + C(high_blood_pressure)
        
- We can create this model using smf.ols()

- OLS stands for simple linear least squares regression

- Two components: the formula and the data are explicitly stated.

- The terms in the formula are columns in the pandas data frame. Easy!

In [3]:
# Declares the model
mod = smf.ols(formula='time ~ C(diabetes) + C(high_blood_pressure)', data=df)

In [4]:
# Fits the model (find the optimal coefficients, adding a random seed ensures consistency)
res = mod.fit()

In [5]:
# Print thes summary output provided by the library.
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   time   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     6.097
Date:                Fri, 05 Nov 2021   Prob (F-statistic):            0.00254
Time:                        11:11:55   Log-Likelihood:                -1718.9
No. Observations:                 299   AIC:                             3444.
Df Residuals:                     296   BIC:                             3455.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                     139.3851      6.658     20.934      0.000     126.282     152.489
C(diabetes)[T.1]                4.9059      8.949      0.548      0.584     -12.706      22.518
C(high_blood_pressure)[T.1]   -31.8228      9.247     -3.441      0.001     -50.021     -13.624
==============================================================================
Omnibus:                      159.508   Durbin-Watson:                   0.076
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               18.166
Skew:                           0.076   Prob(JB):                     0.000114
Kurtosis:                       1.802   Cond. No.                         2.82
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### A lot of useful information is provided by default.

- The dependent variable : time (number of days at the hospital)
- Method: The type of model that was fitted (OLS)
- Nb observations: The number of datapoints (299 patients)
- R2: The fraction of explained variance
- A list of predictors
- For each predictor: coefficient, standard error of the coefficients, p-value, 95% confidence intervals. We can see that only high blood pressure is a significant predictor (p = 0.001), while diabetes is not (0.584).
- Warnings if there are numerical issues (hopefully not!)

### We can now interpret the learned model

$time = 139.3851 + 4.9059 * diabetes + (-31.8228) * highBloodPressure$

- What is the expected time if the patient does not have diabetes or high blood pressure?
- What if he doesn't have diabetes and he has high blood pressure?
- What if he has both?

1. People who do not have diabetes or high blood pressure stay in the hospital for an average of 139 days
2. People who have diabetes but do not have blood pressure stay 139 + 4.9 days ~ 144 days
3. People who do not have diabetes but have blood pressure stay 139 - 31 days ~ 108 days
4. People with diabetes and blood pressure stay 139 + 4.9 - 31.8 ~ 112 days

- How else could we calculate that?

In [6]:
selected = df.loc[(df['diabetes'] == 0) & (df["high_blood_pressure"] == 0)]

selected['time'].mean()

139.0

- Nije li čudno da visoki tlak ima negativan koeficijent? Čini se da pacijenti s visokim tlakom ostaju u bolnici kraći broj dana, iako bi se očekivalo suprotno. Znali li netko možda zašto bi to bilo tako?

# Linearna regresija: Modeliranje dana provedenih u bolnici V2

- Jedan od razloga zašto se ozbiljna stanja mogu povezati s kraćim vremenom provedenim u bolnici je treći čimbenik: smrt 💀. Pacijenti koji imaju ozbiljno stanje mogli bi provesti manje vremena u bolnici jer umiru.

- Dobijmo bolji osjećaj što se događa modelirajući vrijeme provedeno u bolnici sa smrću kao prediktorom.

- Ovaj put ćemo dodati interakcijske značajke. 

In [7]:
# we use a*b to add terms: a, b, a:b, and intercept

mod = smf.ols(formula='time ~ C(high_blood_pressure) * C(DEATH_EVENT,  Treatment(reference=0)) + C(diabetes)',
              data=df)

# C(DEATH_EVENT,  Treatment(reference=0)) implies that we are considering the population that did not die!

res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   time   R-squared:                       0.303
Model:                            OLS   Adj. R-squared:                  0.293
Method:                 Least Squares   F-statistic:                     31.92
Date:                Fri, 05 Nov 2021   Prob (F-statistic):           4.32e-22
Time:                        11:11:55   Log-Likelihood:                -1671.0
No. Observations:                 299   AIC:                             3352.
Df Residuals:                     294   BIC:                             3371.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===========================================================================================================================================
                                                                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                 164.8348      6.476     25.452      0.000     152.089     177.581
C(high_blood_pressure)[T.1]                                               -26.1462      9.781     -2.673      0.008     -45.395      -6.897
C(DEATH_EVENT, Treatment(reference=0))[T.1]                               -86.4520     10.286     -8.405      0.000    -106.696     -66.208
C(diabetes)[T.1]                                                            4.7903      7.655      0.626      0.532     -10.275      19.855
C(high_blood_pressure)[T.1]:C(DEATH_EVENT, Treatment(reference=0))[T.1]     2.7778     16.725      0.166      0.868     -30.137      35.693
==============================================================================
Omnibus:                       34.161   Durbin-Watson:                   0.484
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               11.463
Skew:                           0.185   Prob(JB):                      0.00324
Kurtosis:                       2.115   Cond. No.                         6.31
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Interpretation

- This model lets us see that death is associated with a lower number of days spent at the hospital.
- Notice how R2 is much larger compared to the previous model: more variance in the data is explained.
- Those that have high blood pressure stay for less time (-26 days on average), those who have blood pressure __and__ die spend 2.7 days more hospitalized on average, although this is not statistically significant.

# Standardization

In [8]:
formula = 'time ~ age + C(high_blood_pressure)'
mod = smf.ols(formula=formula, data=df)
res = mod.fit()
res.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   time   R-squared:                       0.081
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     13.10
Date:                Fri, 05 Nov 2021   Prob (F-statistic):           3.55e-06
Time:                        11:11:55   Log-Likelihood:                -1712.3
No. Observations:                 299   AIC:                             3431.
Df Residuals:                     296   BIC:                             3442.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                     222.7404     22.559      9.874      0.000     178.343     267.137
C(high_blood_pressure)[T.1]   -28.7444      9.083     -3.165      0.002     -46.620     -10.869
age                            -1.3543      0.365     -3.709      0.000      -2.073      -0.636
==============================================================================
Omnibus:                       99.161   Durbin-Watson:                   0.158
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               15.705
Skew:                           0.063   Prob(JB):                     0.000389
Kurtosis:                       1.884   Cond. No.                         324.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

What can we conclude from the intercept?

> that patients who do not have high blood pressure and are zero years remain in the hospital for ~ 223 days

In [9]:
# how we standardize the countinuous variables
columns_to_standardize = [
    "age",
    "creatinine_phosphokinase",
    "ejection_fraction",
    "platelets",
    "serum_creatinine",
    "serum_sodium"
]

for col in columns_to_standardize:
    df[col] = (df[col] - df[col].mean()) / df[col].std()  # standardize column


In [10]:
formula = 'time ~ age + C(high_blood_pressure)'
mod = smf.ols(formula=formula, data=df)
res = mod.fit()
res.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   time   R-squared:                       0.081
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     13.10
Date:                Fri, 05 Nov 2021   Prob (F-statistic):           3.55e-06
Time:                        11:11:55   Log-Likelihood:                -1712.3
No. Observations:                 299   AIC:                             3431.
Df Residuals:                     296   BIC:                             3442.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                     140.3550      5.367     26.150      0.000     129.792     150.918
C(high_blood_pressure)[T.1]   -28.7444      9.083     -3.165      0.002     -46.620     -10.869
age                           -16.1088      4.343     -3.709      0.000     -24.656      -7.562
==============================================================================
Omnibus:                       99.161   Durbin-Watson:                   0.158
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               15.705
Skew:                           0.063   Prob(JB):                     0.000389
Kurtosis:                       1.884   Cond. No.                         2.43
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

What can we conclude from the intercept now?

> that patients who do not have high blood pressure and have an average number of years remain in the hospital for ~ 140 days

# Logaritamska transformacija

In [11]:
df = pd.read_csv('data/heart_failure_clinical_records_dataset.csv')

df["logtime"] = np.log(df["time"])
mod = smf.ols(formula='logtime ~ C(high_blood_pressure) * C(DEATH_EVENT,  Treatment(reference=0)) + C(diabetes)',
              data=df)
res = mod.fit()
res.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                logtime   R-squared:                       0.360
Model:                            OLS   Adj. R-squared:                  0.351
Method:                 Least Squares   F-statistic:                     41.29
Date:                Fri, 05 Nov 2021   Prob (F-statistic):           1.86e-27
Time:                        11:11:55   Log-Likelihood:                -325.33
No. Observations:                 299   AIC:                             660.7
Df Residuals:                     294   BIC:                             679.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===========================================================================================================================================
                                                                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                   4.9759      0.072     69.206      0.000       4.834       5.117
C(high_blood_pressure)[T.1]                                                -0.1872      0.109     -1.724      0.086      -0.401       0.026
C(DEATH_EVENT, Treatment(reference=0))[T.1]                                -1.0647      0.114     -9.323      0.000      -1.289      -0.840
C(diabetes)[T.1]                                                            0.0715      0.085      0.841      0.401      -0.096       0.239
C(high_blood_pressure)[T.1]:C(DEATH_EVENT, Treatment(reference=0))[T.1]    -0.1131      0.186     -0.609      0.543      -0.479       0.252
==============================================================================
Omnibus:                       24.038   Durbin-Watson:                   0.616
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               29.144
Skew:                          -0.635   Prob(JB):                     4.69e-07
Kurtosis:                       3.851   Cond. No.                         6.31
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

How can we interpret this result now?

In this case, we can say that each unit increase of a variable $ x_n $ multiplies the value of the dependent variable by the exponent of the coefficient of that variable $\beta_n$.

Take the coefficient for diabetes for example (0.0715).

> $ e ^ {0.0715} = 1.074118 $

From this we can say that if everything else stays the same and the value of diabetes increases by one unit value then the patient will stay in the hospital 1.07 times longer, ie 7% longer.